In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

import pathlib
import glob
from copy import deepcopy

import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import jax
import jax.numpy as jnp
import equinox as eqx

In [ ]:
from mc2.utils.data_inspection import (
    get_available_material_names,
    get_file_overview,
    load_and_process_single_from_full_file_overview
)
from mc2.data_management import FrequencySet, MaterialSet, DataSet

## load data from pickle:

In [ ]:
LOAD_ALL_MATERIALS = False
SAVE_SINGLE_PICKLE = False

if LOAD_ALL_MATERIALS:
    dataset = DataSet.load_from_file(pathlib.Path("../../data/processed") / "ten_mat_data.pickle")

    # deleting N49 from dataset for now, since the data is incomplete
    # 50 kHz and 80 kHz are missing
    # 320 kHz has no data at 25 degrees

    available_materials = deepcopy(dataset.material_names)
    print(available_materials)
    print(len(available_materials))

    available_materials.remove("N49")
    print(available_materials)
    print(len(available_materials))
  
    if SAVE_SINGLE_PICKLE:
        dataset_3C90 = dataset.filter_materials(['3C90'])
        dataset_3C90.save_to_file(pathlib.Path("../../data/processed") / "3C90.pickle")  

In [ ]:
LOAD_3C90 = True
dataset = DataSet.load_from_file(pathlib.Path("../../data/processed") / "3C90.pickle")

In [ ]:
# Filter from full dataset for a single material and temperature
data_3C90 = dataset.filter_materials(material_names="3C90")
data_3C90_25_C = data_3C90.filter_temperatures([25])

print("Frequencies:")
for frequency_set in data_3C90_25_C.material_sets[0].frequency_sets:
    print(frequency_set.frequency)

In [ ]:
from mc2.features import add_fe

# Constants
f_sample = 16e6  # 16 MHz
start_sample = 1000
n_periodes = 10

for frequency_set in data_3C90_25_C.material_sets[0].frequency_sets:
    set_b = frequency_set.B
    set_b_and_fe = add_fe(set_b, n_s=200)
    print(set_b.shape)
    print(set_b_and_fe.shape)
    